In [8]:
import gc
import numpy as np
import pandas as pd

from scipy.sparse import coo_matrix, csr_matrix
from tqdm.notebook import tqdm

In [2]:
import os

os.listdir("./data")

['movie_titles.csv',
 'combined_data_4.txt',
 'combined_data_3.txt',
 'qualifying.txt',
 'combined_data_2.txt',
 'probe.txt',
 'combined_data_1.txt',
 'README',
 'archive.zip']

In [3]:
"""A sample of the file format (details in README.md)"""

with open("./data/combined_data_1.txt") as f:
    for i in range(4):
        print(f.readline())
        
# movie id:
# user_id, rating, date

1:

1488844,3,2005-09-06

822109,5,2005-05-13

885013,4,2005-10-19



In [4]:
# read function and transformation are based on this notebook 
# https://www.kaggle.com/sh1ngg/netflix-matrix-factorization-with-h2o4gpu

# reading all data fills my ram :/
files = [
    './data/combined_data_1.txt',
    './data/combined_data_2.txt',
    #'./data/combined_data_3.txt',
    #'./data/combined_data_4.txt',
]

coo_row = []
coo_col = []
coo_val = []

for file in files:
    print(f"processing {file}...")
    with open(file, "r") as f:
        movie = -1
        for line in tqdm(f):
            if line.endswith(':\n'):
                movie = int(line[:-2]) - 1  # why - 1?
                continue
            assert movie >= 0
            splitted = line.split(',')
            user = int(splitted[0])
            rating = float(splitted[1])
            coo_row.append(user)
            coo_col.append(movie)
            coo_val.append(rating)
    gc.collect()

processing ./data/combined_data_1.txt...


0it [00:00, ?it/s]

processing ./data/combined_data_2.txt...


0it [00:00, ?it/s]

In [5]:
# trandformation

coo_val = np.array(coo_val, dtype=np.float32)  # rating
coo_col = np.array(coo_col, dtype=np.int32)  # movie
coo_row = np.array(coo_row)  # user

_, indices = np.unique(coo_row, return_inverse=True)
del _, coo_row

gc.collect()

print(len(coo_val), len(indices), len(coo_col))

51031355 51031355 51031355


In [6]:
coo_movies = coo_matrix((coo_val, (indices, coo_col)))
shape = coo_movies.shape
print('R matrix size', shape)

gc.collect()

R matrix size (478018, 9210)


20

In [12]:
csr_matrix(coo_movies)[0, 0:100]

  (0, 29)	3.0
